In [12]:
import pandas as pd
import pybedtools
import numpy as np
import os
pybedtools.helpers.set_tempdir('/data/projects/temp')

In [13]:
input_path_minus = "/data/private/pdutta/Collab_data/Jim_work/CPD-seq/04-damage-1bp/Tissue_1hr_S2_CPD_1bp_sorted_minus.bed"
input_path_plus = "/data/private/pdutta/Collab_data/Jim_work/CPD-seq/04-damage-1bp/Tissue_1hr_S2_CPD_1bp_sorted_plus.bed"
input_path_noUV_minus = "/data/private/pdutta/Collab_data/Jim_work/CPD-seq/04-damage-1bp/Tissue_noUV_S1_CPD_1bp_sorted_minus.bed"
input_path_noUV_plus = "/data/private/pdutta/Collab_data/Jim_work/CPD-seq/04-damage-1bp/Tissue_noUV_S1_CPD_1bp_sorted_plus.bed"

In [14]:
wgs_data_folder = "/data/private/pdutta/Collab_data/Jim_work/Mouse_WGS/SNV/"

In [15]:
df_plus =pd.read_csv(input_path_plus, sep="\t", header=None)
#df_minus =pd.read_csv(input_path_minus, sep="\t", header=None)
#df_noUV_minus =pd.read_csv(input_path_noUV_minus, sep="\t", header=None)
df_noUV_plus =pd.read_csv(input_path_noUV_plus, sep="\t", header=None)

In [6]:
print(df_plus.shape, df_noUV_plus.shape)

(99495320, 5) (15069500, 5)


In [7]:
df_UV_plus = df_plus.drop_duplicates()
df_UV_plus

,0,1,2,3,4
0,chr1,3050029,3050030,+,AA
1,chr1,3050030,3050031,+,AA
2,chr1,3050054,3050055,+,GG
3,chr1,3050055,3050056,+,GG
4,chr1,3050076,3050077,+,AG
...,...,...,...,...,...
99495301,pUC19,2626,2627,+,GA
99495306,pUC19,2634,2635,+,AA
99495310,pUC19,2635,2636,+,AA
99495314,pUC19,2635,2636,+,AG


In [8]:
df_noUV_plus = df_noUV_plus.drop_duplicates()
df_noUV_plus

,0,1,2,3,4
0,chr1,3050021,3050022,+,GA
1,chr1,3050022,3050023,+,GA
2,chr1,3050413,3050414,+,AG
3,chr1,3050414,3050415,+,AG
4,chr1,3050720,3050721,+,GG
...,...,...,...,...,...
15069474,pUC19,2626,2627,+,GA
15069478,pUC19,2634,2635,+,AA
15069488,pUC19,2635,2636,+,AA
15069498,pUC19,2635,2636,+,AG


In [9]:
df_merged = pd.merge(df_plus, df_noUV_plus, how="outer", indicator=True)
df_merged

,0,1,2,3,4,_merge
0,chr1,3050029,3050030,+,AA,left_only
1,chr1,3050030,3050031,+,AA,left_only
2,chr1,3050054,3050055,+,GG,left_only
3,chr1,3050055,3050056,+,GG,left_only
4,chr1,3050076,3050077,+,AG,left_only
...,...,...,...,...,...,...
111903607,chrY_JH584303v1_random,155795,155796,+,AG,right_only
111903608,chrY_JH584303v1_random,156895,156896,+,GA,right_only
111903609,chrY_JH584303v1_random,156896,156897,+,GA,right_only
111903610,pUC19,1616,1617,+,GA,right_only


In [10]:
df_merged['_merge'].unique()

['left_only', 'both', 'right_only']
Categories (3, object): ['left_only', 'right_only', 'both']

In [11]:
only_in_df_plus = df_merged[df_merged['_merge'] == 'left_only'].drop('_merge', axis=1)
only_in_df_plus

,0,1,2,3,4
0,chr1,3050029,3050030,+,AA
1,chr1,3050030,3050031,+,AA
2,chr1,3050054,3050055,+,GG
3,chr1,3050055,3050056,+,GG
4,chr1,3050076,3050077,+,AG
...,...,...,...,...,...
99327569,chrY_JH584303v1_random,157723,157724,+,AG
99327570,chrY_JH584303v1_random,157983,157984,+,GA
99327571,chrY_JH584303v1_random,157984,157985,+,GA
99327572,chrY_JH584303v1_random,158010,158011,+,GG


In [17]:
only_in_df_plus.drop_duplicates().reset_index(drop=True)

,0,1,2,3,4
0,chr1,3050029,3050030,+,AA
1,chr1,3050030,3050031,+,AA
2,chr1,3050054,3050055,+,GG
3,chr1,3050055,3050056,+,GG
4,chr1,3050076,3050077,+,AG
...,...,...,...,...,...
88928138,chrY_JH584303v1_random,157723,157724,+,AG
88928139,chrY_JH584303v1_random,157983,157984,+,GA
88928140,chrY_JH584303v1_random,157984,157985,+,GA
88928141,chrY_JH584303v1_random,158010,158011,+,GG


In [11]:
only_in_noUV_df_plus = df_merged[df_merged['_merge'] == 'right_only'].drop('_merge', axis=1)
only_in_noUV_df_plus

,0,1,2,3,4
90995852,chr1,3050021,3050022,+,GA
90995853,chr1,3050022,3050023,+,GA
90995854,chr1,3050413,3050414,+,AG
90995855,chr1,3050414,3050415,+,AG
90995856,chr1,3050721,3050722,+,GG
...,...,...,...,...,...
103404139,chrY_JH584303v1_random,155795,155796,+,AG
103404140,chrY_JH584303v1_random,156895,156896,+,GA
103404141,chrY_JH584303v1_random,156896,156897,+,GA
103404142,pUC19,1616,1617,+,GA


In [12]:
both_in_df = df_merged[df_merged['_merge'] == 'both'].drop('_merge', axis=1)
both_in_df

,0,1,2,3,4
53,chr1,3050720,3050721,+,GG
60,chr1,3050881,3050882,+,GG
84,chr1,3051229,3051230,+,AA
108,chr1,3051533,3051534,+,GG
109,chr1,3051534,3051535,+,GG
...,...,...,...,...,...
90995847,pUC19,2626,2627,+,GA
90995848,pUC19,2634,2635,+,AA
90995849,pUC19,2635,2636,+,AA
90995850,pUC19,2635,2636,+,AG


In [13]:
only_in_noUV_df_plus[0].unique()

array(['chr1', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15',
       'chr16', 'chr17', 'chr18', 'chr19', 'chr1_GL456210v1_random',
       'chr1_GL456211v1_random', 'chr1_GL456212v1_random',
       'chr1_GL456221v1_random', 'chr1_GL456239v1_random',
       'chr1_MU069434v1_random', 'chr2', 'chr3', 'chr4',
       'chr4_JH584295v1_random', 'chr5', 'chr5_GL456354v1_random',
       'chr5_JH584296v1_random', 'chr5_JH584297v1_random',
       'chr5_JH584298v1_random', 'chr5_JH584299v1_random', 'chr6', 'chr7',
       'chr7_GL456219v1_random', 'chr8', 'chr9', 'chrM',
       'chrUn_GL456359v1', 'chrUn_GL456360v1', 'chrUn_GL456366v1',
       'chrUn_GL456367v1', 'chrUn_GL456368v1', 'chrUn_GL456370v1',
       'chrUn_GL456372v1', 'chrUn_GL456378v1', 'chrUn_GL456379v1',
       'chrUn_GL456381v1', 'chrUn_GL456382v1', 'chrUn_GL456383v1',
       'chrUn_GL456385v1', 'chrUn_GL456387v1', 'chrUn_GL456389v1',
       'chrUn_GL456390v1', 'chrUn_GL456392v1', 'chrUn_GL456394v1',
       'chrUn_GL456396v1', 

In [14]:
only_in_noUV_df_plus.groupby([0]).size()

0
chr1                      912599
chr10                     604646
chr11                     642427
chr12                     564011
chr13                     570294
                           ...  
chrY_JH584300v1_random       480
chrY_JH584301v1_random       530
chrY_JH584302v1_random       362
chrY_JH584303v1_random       369
pUC19                          2
Length: 62, dtype: int64

# WGS data (SNV)

In [7]:
combined_df = pd.DataFrame()
for filename in os.listdir(wgs_data_folder):
    # Check if the file is a txt file
    if filename.endswith('.txt'):
        # Construct the full filepath
        filepath = os.path.join(wgs_data_folder, filename)
        print(filepath)
        
        # Read the file into a DataFrame
        temp_df = pd.read_csv(filepath, delimiter='\t', header=None)  # Assuming tab-delimited, change as needed
        
        
        # Append the data to the combined DataFrame
        combined_df = pd.concat([combined_df, temp_df], ignore_index=True)

# Show the combined DataFrame

/data/private/pdutta/Collab_data/Jim_work/Mouse_WGS/SNV/15_seqinfo.txt
/data/private/pdutta/Collab_data/Jim_work/Mouse_WGS/SNV/Y_seqinfo.txt
/data/private/pdutta/Collab_data/Jim_work/Mouse_WGS/SNV/10_seqinfo.txt
/data/private/pdutta/Collab_data/Jim_work/Mouse_WGS/SNV/2_seqinfo.txt
/data/private/pdutta/Collab_data/Jim_work/Mouse_WGS/SNV/18_seqinfo.txt
/data/private/pdutta/Collab_data/Jim_work/Mouse_WGS/SNV/19_seqinfo.txt
/data/private/pdutta/Collab_data/Jim_work/Mouse_WGS/SNV/13_seqinfo.txt
/data/private/pdutta/Collab_data/Jim_work/Mouse_WGS/SNV/1_seqinfo.txt
/data/private/pdutta/Collab_data/Jim_work/Mouse_WGS/SNV/3_seqinfo.txt
/data/private/pdutta/Collab_data/Jim_work/Mouse_WGS/SNV/5_seqinfo.txt
/data/private/pdutta/Collab_data/Jim_work/Mouse_WGS/SNV/8_seqinfo.txt
/data/private/pdutta/Collab_data/Jim_work/Mouse_WGS/SNV/12_seqinfo.txt
/data/private/pdutta/Collab_data/Jim_work/Mouse_WGS/SNV/6_seqinfo.txt
/data/private/pdutta/Collab_data/Jim_work/Mouse_WGS/SNV/9_seqinfo.txt
/data/private/

In [8]:
combined_df

,0,1,2,3,4
0,S319_final_vep_snpfiltered,15,3066686,N:AC[C>T]GT,-1
1,S319_final_vep_snpfiltered,15,3391439,T:GG[C>T]GG,1
2,S319_final_vep_snpfiltered,15,3911756,N:TC[C>T]TT,1
3,S319_final_vep_snpfiltered,15,3930627,N:AT[C>T]CC,1
4,S319_final_vep_snpfiltered,15,4574999,T:CC[C>T]TT,-1
...,...,...,...,...,...
828281,S346_final_vep_snpfiltered,X,168111504,N:CT[C>T]TT,1
828282,S346_final_vep_snpfiltered,X,168232166,N:CT[C>T]TT,1
828283,S346_final_vep_snpfiltered,X,168273413,N:AA[C>T]TT,1
828284,S346_final_vep_snpfiltered,X,168326651,N:TC[C>T]TT,-1


In [9]:
combined_df[['ID', '0']] = combined_df[0].str.split('_', 1, expand=True)

# Drop the original column
combined_df.drop(columns=[0], inplace=True)

# Reorder the columns to make new_col1 and new_col2 the first and second columns
combined_df = combined_df[['ID', '0'] + [col for col in combined_df.columns if col not in ['ID', '0']]]
combined_df['ID'] = combined_df['ID'].str.replace('S','').astype(int)
combined_df

/data/projects/temp/ipykernel_1922124/4152682433.py:1: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  combined_df[['ID', '0']] = combined_df[0].str.split('_', 1, expand=True)


,ID,0,1,2,3,4
0,319,final_vep_snpfiltered,15,3066686,N:AC[C>T]GT,-1
1,319,final_vep_snpfiltered,15,3391439,T:GG[C>T]GG,1
2,319,final_vep_snpfiltered,15,3911756,N:TC[C>T]TT,1
3,319,final_vep_snpfiltered,15,3930627,N:AT[C>T]CC,1
4,319,final_vep_snpfiltered,15,4574999,T:CC[C>T]TT,-1
...,...,...,...,...,...,...
828281,346,final_vep_snpfiltered,X,168111504,N:CT[C>T]TT,1
828282,346,final_vep_snpfiltered,X,168232166,N:CT[C>T]TT,1
828283,346,final_vep_snpfiltered,X,168273413,N:AA[C>T]TT,1
828284,346,final_vep_snpfiltered,X,168326651,N:TC[C>T]TT,-1


In [10]:
df_interest = combined_df[(combined_df["ID"] >= 332) & (combined_df["ID"] <= 346)].reset_index(drop=True)
df_interest[1] = df_interest[1].astype(str)
df_interest = df_interest.drop_duplicates()
df_interest

,ID,0,1,2,3,4
0,334,final_vep_snpfiltered,15,3058069,N:CC[C>T]TA,-1
1,334,final_vep_snpfiltered,15,3063770,N:AA[T>A]AT,-1
2,334,final_vep_snpfiltered,15,3080731,N:GT[C>T]TG,1
3,334,final_vep_snpfiltered,15,3094450,N:AG[T>G]TT,1
4,334,final_vep_snpfiltered,15,3109676,N:AC[C>T]CA,1
...,...,...,...,...,...,...
421491,346,final_vep_snpfiltered,X,168111504,N:CT[C>T]TT,1
421492,346,final_vep_snpfiltered,X,168232166,N:CT[C>T]TT,1
421493,346,final_vep_snpfiltered,X,168273413,N:AA[C>T]TT,1
421494,346,final_vep_snpfiltered,X,168326651,N:TC[C>T]TT,-1


In [11]:
# Custom sorting function
def custom_chromosome_sort(chromosome):
    if chromosome.isdigit():
        return int(chromosome)
    if chromosome == 'X':
        return 23
    if chromosome == 'Y':
        return 24
    return 25  # for any other value that might appear

In [12]:
# Apply custom sorting function
df_interest['sort_key'] = df_interest[1].apply(custom_chromosome_sort)
df_interest = df_interest.sort_values(by=['sort_key']).drop(columns=['sort_key']).reset_index(drop=True)
df_interest

,ID,0,1,2,3,4
0,334,final_vep_snpfiltered,1,17162732,U:TT[T>A]TT,-1
1,334,final_vep_snpfiltered,1,130607017,N:TC[T>C]AT,-1
2,334,final_vep_snpfiltered,1,130613091,N:TT[C>T]AT,-1
3,334,final_vep_snpfiltered,1,130653834,T:AC[C>T]TT,1
4,334,final_vep_snpfiltered,1,130674696,N:TT[C>T]TG,1
...,...,...,...,...,...,...
412068,334,final_vep_snpfiltered,X,112617823,T:AT[C>T]AG,-1
412069,334,final_vep_snpfiltered,X,112578837,T:AT[C>T]CT,-1
412070,334,final_vep_snpfiltered,X,112683167,T:CC[C>A]GG,-1
412071,346,final_vep_snpfiltered,X,168621252,U:TT[C>T]TG,1


In [13]:
df_interest[3]

0         U:TT[T>A]TT
1         N:TC[T>C]AT
2         N:TT[C>T]AT
3         T:AC[C>T]TT
4         N:TT[C>T]TG
             ...     
412068    T:AT[C>T]AG
412069    T:AT[C>T]CT
412070    T:CC[C>A]GG
412071    U:TT[C>T]TG
412072    N:TC[T>C]TG
Name: 3, Length: 412073, dtype: object

In [14]:
df_interest['type'] = df_interest[3].apply(lambda x: x.split(":")[0])
df_interest

,ID,0,1,2,3,4,type
0,334,final_vep_snpfiltered,1,17162732,U:TT[T>A]TT,-1,U
1,334,final_vep_snpfiltered,1,130607017,N:TC[T>C]AT,-1,N
2,334,final_vep_snpfiltered,1,130613091,N:TT[C>T]AT,-1,N
3,334,final_vep_snpfiltered,1,130653834,T:AC[C>T]TT,1,T
4,334,final_vep_snpfiltered,1,130674696,N:TT[C>T]TG,1,N
...,...,...,...,...,...,...,...
412068,334,final_vep_snpfiltered,X,112617823,T:AT[C>T]AG,-1,T
412069,334,final_vep_snpfiltered,X,112578837,T:AT[C>T]CT,-1,T
412070,334,final_vep_snpfiltered,X,112683167,T:CC[C>A]GG,-1,T
412071,346,final_vep_snpfiltered,X,168621252,U:TT[C>T]TG,1,U


In [15]:
df_interest['type'].unique()

array(['U', 'N', 'T', 'B'], dtype=object)

## Found four type of tags  
   * __N__: This might stand for "Normal," perhaps indicating that the sequence comes from normal tissue or represents a reference sequence.
   * __T__: This could stand for "Tumor," possibly indicating that the sequence comes from tumor tissue, especially if you're working on cancer genomics data.
   * __U__: This could stand for "Unknown," perhaps indicating that the origin of the sequence or the quality of the data is uncertain.
   * __B__: This is not standard in genomics as far as my training data goes. It could be specific to the particular dataset or pipeline you're working with. It might stand for something like "Baseline" or "Both" (if it's a combined analysis of tumor and normal, for example), but that's purely speculative.

In [16]:
df_interest.groupby('type').size()

type
B       912
N    271852
T     55255
U     84054
dtype: int64

In [17]:
df_interest_bed = pd.DataFrame()
df_interest_bed['chr'] = 'chr'+ df_interest[1].astype(str)
df_interest_bed['start'] = df_interest[2]-1
df_interest_bed['end'] = df_interest[2]
df_interest_bed['ID'] = df_interest['ID']
df_interest_bed['SNV'] = df_interest[3]
df_interest_bed

,chr,start,end,ID,SNV
0,chr1,17162731,17162732,334,U:TT[T>A]TT
1,chr1,130607016,130607017,334,N:TC[T>C]AT
2,chr1,130613090,130613091,334,N:TT[C>T]AT
3,chr1,130653833,130653834,334,T:AC[C>T]TT
4,chr1,130674695,130674696,334,N:TT[C>T]TG
...,...,...,...,...,...
412068,chrX,112617822,112617823,334,T:AT[C>T]AG
412069,chrX,112578836,112578837,334,T:AT[C>T]CT
412070,chrX,112683166,112683167,334,T:CC[C>A]GG
412071,chrX,168621251,168621252,346,U:TT[C>T]TG


In [18]:
df_interest_bed.drop_duplicates()

,chr,start,end,ID,SNV
0,chr1,17162731,17162732,334,U:TT[T>A]TT
1,chr1,130607016,130607017,334,N:TC[T>C]AT
2,chr1,130613090,130613091,334,N:TT[C>T]AT
3,chr1,130653833,130653834,334,T:AC[C>T]TT
4,chr1,130674695,130674696,334,N:TT[C>T]TG
...,...,...,...,...,...
412068,chrX,112617822,112617823,334,T:AT[C>T]AG
412069,chrX,112578836,112578837,334,T:AT[C>T]CT
412070,chrX,112683166,112683167,334,T:CC[C>A]GG
412071,chrX,168621251,168621252,346,U:TT[C>T]TG


## Generate multiple UV_plus CPD_seq data(by increasing the number of nucleotides in both starting and end position) and fond the intersection with the C>T mutation data

In [24]:
# Create a list of chromosomes to keep
chromosomes_to_keep = ['chr' + str(i) for i in range(1, 23)] + ['chrX', 'chrY']

# Filter the original DataFrame to only include specified chromosomes
df_UV_plus_interested = df_UV_plus[df_UV_plus[0].isin(chromosomes_to_keep)].reset_index(drop=True)
print(df_UV_plus_interested.shape, df_interest_bed.shape)

# Create a BedTool object from another DataFrame
bd_interest_SNV_bed = pybedtools.BedTool.from_dataframe(df_interest_bed)

# Initialize an empty DataFrame for temporary use
df_temp = pd.DataFrame()
df_temp['chr_name'] = df_UV_plus_interested[0]

# Convert df_noUV_plus_interested to a BedTool object initially
df_UV_plus_bed = pybedtools.BedTool.from_dataframe(df_UV_plus_interested)

for i in range(51):
    # Populate df_temp with updated start and end positions
    df_temp['chr_start'] = df_UV_plus_interested[1] - i
    df_temp['chr_end'] = df_UV_plus_interested[2] + i
    print("The number of SNV counts {} at {} epoch".format(bd_interest_SNV_bed.count(), i))
    # Convert df_temp to a BedTool object
    bd_UV_plus_bed = pybedtools.BedTool.from_dataframe(df_temp)
    # Perform the intersection
    intersect_CPD_SNV = bd_UV_plus_bed.intersect(bd_interest_SNV_bed, wa=True, wb=True)
    # Convert the result back to a DataFrame and set column names
    df_intersect_CPD_SNV = intersect_CPD_SNV.to_dataframe()
    df_intersect_CPD_SNV.columns = ['chr_name', 'CPD_start', 'CPD_end', 'chr_name_SNV', 'SNV_start', 'SNV_end', 'ID','SNV_Info']
    #print(df_intersect_noCPD_SNV[['chr_name_SNV', 'SNV_start', 'SNV_end', 'ID','SNV_Info']].drop_duplicates())
    
    # Save the result to a file
    df_intersect_CPD_SNV.to_csv(f"/data/private/pdutta/Collab_data/Jim_work/Output/UV_plus/intersected_CPD_SNV_UV_plus_{i}.tsv", sep='\t', index=False)
    
    # Identify intersecting regions for removal in the next iteration
    intersecting_SNV_regions = bd_interest_SNV_bed.intersect(bd_UV_plus_bed, wa=True, u=True)
    print("The intersecting regions size is :",intersecting_SNV_regions.to_dataframe().shape)
    #print(bd_interest_SNV_bed.to_dataframe())
    
    # Remove intersecting regions for the next iteration
    bd_interest_SNV_bed = bd_interest_SNV_bed.subtract(intersecting_SNV_regions)
    #print(bd_interest_SNV_bed.to_dataframe())
    print("After removal of intersecting region the remaining SNVs are ", bd_interest_SNV_bed.count())
    #df_noUV_plus_interested.columns = ['chr_name', 'CPD_start', 'CPD_end']
    
    print("#############################################################")

(90787037, 5) (412073, 5)
The number of SNV counts 412073 at 0 epoch
The intersecting regions size is : (23933, 5)
After removal of intersecting region the remaining SNVs are  388140
#############################################################
The number of SNV counts 388140 at 1 epoch
The intersecting regions size is : (14631, 5)
After removal of intersecting region the remaining SNVs are  373509
#############################################################
The number of SNV counts 373509 at 2 epoch
The intersecting regions size is : (12562, 5)
After removal of intersecting region the remaining SNVs are  360947
#############################################################
The number of SNV counts 360947 at 3 epoch
The intersecting regions size is : (10613, 5)
After removal of intersecting region the remaining SNVs are  350334
#############################################################
The number of SNV counts 350334 at 4 epoch
The intersecting regions size is : (10100, 5)
After rem

## Generate multiple NoUV_plus CPD_seq data(by increasing the number of nucleotides in both starting and end position) and fond the intersection with the C>T mutation data

In [83]:
# Create a list of chromosomes to keep
chromosomes_to_keep = ['chr' + str(i) for i in range(1, 23)] + ['chrX', 'chrY']

# Filter the original DataFrame to only include specified chromosomes
df_noUV_plus_interested = df_noUV_plus[df_noUV_plus[0].isin(chromosomes_to_keep)].reset_index(drop=True)
print(df_noUV_plus_interested.shape, df_interest_bed.shape)
input()
# Create a BedTool object from another DataFrame
bd_interest_SNV_bed = pybedtools.BedTool.from_dataframe(df_interest_bed)

# Initialize an empty DataFrame for temporary use
df_temp = pd.DataFrame()
df_temp['chr_name'] = df_noUV_plus_interested[0]

# Convert df_noUV_plus_interested to a BedTool object initially
df_noUV_plus_bed = pybedtools.BedTool.from_dataframe(df_noUV_plus_interested)

for i in range(51):
    # Populate df_temp with updated start and end positions
    df_temp['chr_start'] = df_noUV_plus_interested[1] - i
    df_temp['chr_end'] = df_noUV_plus_interested[2] + i
    print("The number of SNV counts {} at {} epoch".format(bd_interest_SNV_bed.count(), i))
    # Convert df_temp to a BedTool object
    bd_noUV_plus_bed = pybedtools.BedTool.from_dataframe(df_temp)
    # Perform the intersection
    intersect_noCPD_SNV = bd_noUV_plus_bed.intersect(bd_interest_SNV_bed, wa=True, wb=True)
    # Convert the result back to a DataFrame and set column names
    df_intersect_noCPD_SNV = intersect_noCPD_SNV.to_dataframe()
    df_intersect_noCPD_SNV.columns = ['chr_name', 'CPD_start', 'CPD_end', 'chr_name_SNV', 'SNV_start', 'SNV_end', 'ID','SNV_Info']
    #print(df_intersect_noCPD_SNV[['chr_name_SNV', 'SNV_start', 'SNV_end', 'ID','SNV_Info']].drop_duplicates())
    
    # Save the result to a file
    df_intersect_noCPD_SNV.to_csv(f"/data/private/pdutta/Collab_data/Jim_work/Output/No_UV_Plus/intersected_CPD_SNV_noUV_plus_{i}.tsv", sep='\t', index=False)
    
    # Identify intersecting regions for removal in the next iteration
    intersecting_SNV_regions = bd_interest_SNV_bed.intersect(bd_noUV_plus_bed, wa=True, u=True)
    print("The intersecting regions size is :",intersecting_SNV_regions.to_dataframe().shape)
    #print(bd_interest_SNV_bed.to_dataframe())
    
    # Remove intersecting regions for the next iteration
    bd_interest_SNV_bed = bd_interest_SNV_bed.subtract(intersecting_SNV_regions)
    #print(bd_interest_SNV_bed.to_dataframe())
    print("After removal of intersecting region the remaining SNVs are ", bd_interest_SNV_bed.count())
    #df_noUV_plus_interested.columns = ['chr_name', 'CPD_start', 'CPD_end']
    
    print("#############################################################")


(14424047, 5) (412073, 5)


The number of SNV counts 412073 at 0 epoch
The intersecting regions size is : (4055, 5)
After removal of intersecting region the remaining SNVs are  408018
#############################################################
The number of SNV counts 408018 at 1 epoch
The intersecting regions size is : (2719, 5)
After removal of intersecting region the remaining SNVs are  405299
#############################################################
The number of SNV counts 405299 at 2 epoch
The intersecting regions size is : (2540, 5)
After removal of intersecting region the remaining SNVs are  402759
#############################################################
The number of SNV counts 402759 at 3 epoch
The intersecting regions size is : (2243, 5)
After removal of intersecting region the remaining SNVs are  400516
#############################################################
The number of SNV counts 400516 at 4 epoch
The intersecting regions size is : (2029, 5)
After removal of intersecting region the

In [34]:
chromosomes_to_keep = ['chr' + str(i) for i in range(1, 23)] + ['chrX', 'chrY']
df_noUV_plus_interested = df_noUV_plus[df_noUV_plus[0].isin(chromosomes_to_keep)].reset_index(drop=True)
df_interest_SNV_bed = pybedtools.BedTool.from_dataframe(df_interest_bed)
print(df_noUV_plus_interested.shape, df_interest_bed.shape)
df_temp=pd.DataFrame()
df_temp['chr_name'] = df_noUV_plus_interested[0]
for i in range(11):
    df_temp['chr_start'] = df_noUV_plus_interested[1]- i
    df_temp['chr_end'] = df_noUV_plus_interested[2]+ i
    print(df_temp)
    df_UV_plus_bed = pybedtools.BedTool.from_dataframe(df_temp)
    intersect_SNV_CPD = df_UV_plus_bed.intersect(df_interest_SNV_bed, wa=True, wb=True)
    intersect_SNV_CPD_df= intersect_SNV_CPD.to_dataframe()
    intersect_SNV_CPD_df.columns = ['chr_name', 'CPD_start', 'CPD_end', 'chr_name_SNV', 'SNV_start', 'SNV_end']
    print(intersect_SNV_CPD_df)
    intersect_SNV_CPD_df.to_csv("/data/private/pdutta/Collab_data/Jim_work/Output/No_UV_Plus/intersected_CPD_SNV_noUV_plus_"+str(i)+".tsv", sep="\t", index=False)
    print("#############################################################")

(14424047, 5) (421496, 3)
         chr_name  chr_start   chr_end
0            chr1    3050021   3050022
1            chr1    3050022   3050023
2            chr1    3050413   3050414
3            chr1    3050414   3050415
4            chr1    3050720   3050721
...           ...        ...       ...
14424042     chrY   90855678  90855679
14424043     chrY   90855821  90855822
14424044     chrY   90855822  90855823
14424045     chrY   90855830  90855831
14424046     chrY   90855831  90855832

[14424047 rows x 3 columns]
     chr_name  CPD_start    CPD_end chr_name_SNV  SNV_start    SNV_end
0        chr1    4306970    4306971         chr1    4306970    4306971
1        chr1    5857620    5857621         chr1    5857620    5857621
2        chr1    7532894    7532895         chr1    7532894    7532895
3        chr1   10057967   10057968         chr1   10057967   10057968
4        chr1   11061577   11061578         chr1   11061577   11061578
...       ...        ...        ...          ...   

In [65]:
df_filtered_noUV_plus_only.head(20)

,0,1,2,3,4
0,chr1,3050021,3050022,+,GA
1,chr1,3050022,3050023,+,GA
2,chr1,3050413,3050414,+,AG
3,chr1,3050414,3050415,+,AG
4,chr1,3050721,3050722,+,GG
5,chr1,3050756,3050757,+,GA
6,chr1,3050757,3050758,+,GA
7,chr1,3050807,3050808,+,GA
8,chr1,3050808,3050809,+,GA
9,chr1,3050842,3050843,+,AA


## Finding Intersection using `PyBedtools`

In [66]:
df_interest_SNV_bed = pybedtools.BedTool.from_dataframe(df_interest_bed)
df_noUV_plus_only_bed = pybedtools.BedTool.from_dataframe(df_filtered_noUV_plus_only)
intersect_SNV_CPD = df_interest_SNV_bed.intersect(df_noUV_plus_only_bed, wa=True, wb=True)
intersect_SNV_CPD.to_dataframe()

,chrom,start,end,name,score,strand,thickStart,thickEnd
0,chr15,3895294,3895295,chr15,3895294,3895295,+,GG
1,chr15,5579415,5579416,chr15,5579415,5579416,+,AG
2,chr15,6191775,6191776,chr15,6191775,6191776,+,GA
3,chr15,6207261,6207262,chr15,6207261,6207262,+,GA
4,chr15,9691374,9691375,chr15,9691374,9691375,+,GG
...,...,...,...,...,...,...,...,...
3501,chrX,147720411,147720412,chrX,147720411,147720412,+,GG
3502,chrX,39568936,39568937,chrX,39568936,39568937,+,GA
3503,chrX,88746130,88746131,chrX,88746130,88746131,+,GG
3504,chrX,104886995,104886996,chrX,104886995,104886996,+,GG
